In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def get_content(content_url):
    req = requests.get(url=content_url)
    req.encoding = 'utf-8'
    html = req.text
    bs = BeautifulSoup(html, 'lxml')
    
    return bs

In [31]:
def get_reviews(bs):
    entries = bs.find_all('entry')
    reviews = []
    for entry in entries:
        entry_updated = entry.updated.text
        entry_title = entry.title.text
        content = entry.find('content', type='text')
        entry_content_001 = content.text
        entry_im_ratting = entry.find('im:rating').text
        entry_user_name = entry.find('name').text
        item = [entry_updated, entry_title,
                entry__content__001, entry_im_ratting,
                entry__user__name]
        reviews.append(item)
    return reviews

In [32]:
pref = 'https://itunes.apple.com/us/rss/customerreviews/page='
suff = '/id=350189835/sortBy=mostrecent/xml'
# target = 'https://itunes.apple.com/us/rss/customerreviews/page=1/id=350189835/sortBy=mostrecent/xml'
reviews = []
for page in range(1,11):
    target = pref + str(page) + suff
    bs = get_content(target)
    reviews.extend(get_reviews(bs))

In [37]:
idxes = ['entry__updated',
         'entry__title',
         'entry__content__001__#Text',
         'entry__im:rating',
         'entry__user__name'
        ]
df = pd.DataFrame(reviews)
columns = {idx: name for idx, name in enumerate(idxes)}
df.rename(columns=columns, inplace=True)
df.loc[0:249,:].to_csv('weedmaps.csv')